In [84]:
from pymongo import MongoClient
from bson.son import SON
import sqlite3

# SQL PART

In [85]:
conn = sqlite3.connect('northwind.db')
conn.text_factory = lambda x: str(x, 'latin1')
# Use to solve encoding issue

### Exercise 6.1

In [86]:
query1 = """SELECT SubQuery.OrderID, Products.ProductID, Products.ProductName
           FROM Products INNER JOIN (
               SELECT [Order Details].ProductID AS ProductID, [Order Details].OrderID AS OrderID
               FROM [Order Details] INNER JOIN Orders
               ON [Order Details].OrderID = Orders.OrderID
               WHERE Orders.CustomerID = 'ALFKI'
           ) AS SubQuery
           ON Products.ProductID = SubQuery.ProductID ;"""

In [87]:
for row in conn.execute(query1):
    print(row)

(10643, 28, 'Rössle Sauerkraut')
(10643, 39, 'Chartreuse verte')
(10643, 46, 'Spegesild')
(10692, 63, 'Vegie-spread')
(10702, 3, 'Aniseed Syrup')
(10702, 76, 'Lakkalikööri')
(10835, 59, 'Raclette Courdavault')
(10835, 77, 'Original Frankfurter grüne Soße')
(10952, 6, "Grandma's Boysenberry Spread")
(10952, 28, 'Rössle Sauerkraut')
(11011, 58, 'Escargots de Bourgogne')
(11011, 71, 'Flotemysost')


### Exercise 6.2

In [88]:
query2 = """WITH Customer_Orders AS (
               SELECT SubQuery.OrderID, Products.ProductID, Products.ProductName
               FROM Products INNER JOIN (
                   SELECT [Order Details].ProductID AS ProductID, [Order Details].OrderID AS OrderID
                   FROM [Order Details] INNER JOIN Orders
                   ON [Order Details].OrderID = Orders.OrderID
                   WHERE Orders.CustomerID = 'ALFKI'
                   ) AS SubQuery
               ON Products.ProductID = SubQuery.ProductID 
            )
            
            SELECT Customer_Orders.*
            FROM Customer_orders INNER JOIN ( 
                SELECT OrderID, COUNT(*) AS FREQ
                FROM (
                    SELECT OrderID, ProductName
                    FROM Customer_Orders
                    GROUP BY OrderID, ProductName
                    ) 
                GROUP BY OrderID
                ) AS Order_Frequency
            ON Customer_Orders.OrderID = Order_Frequency.OrderID
            WHERE Order_Frequency.FREQ > 1;
            """

In [89]:
for row in conn.execute(query2):
    print(row)

(10643, 28, 'Rössle Sauerkraut')
(10643, 39, 'Chartreuse verte')
(10643, 46, 'Spegesild')
(10702, 3, 'Aniseed Syrup')
(10702, 76, 'Lakkalikööri')
(10835, 59, 'Raclette Courdavault')
(10835, 77, 'Original Frankfurter grüne Soße')
(10952, 6, "Grandma's Boysenberry Spread")
(10952, 28, 'Rössle Sauerkraut')
(11011, 58, 'Escargots de Bourgogne')
(11011, 71, 'Flotemysost')


### Exercise 6.3

In [90]:
query3 = """SELECT OrderID, ProductID, ProductName, Products.UnitPrice*Quantity, TotalPrice
            FROM Orders
            JOIN [Order Details] USING(OrderID)
            JOIN Products USING(ProductID)
            JOIN (SELECT OrderID, SUM(UnitPrice*Quantity) as TotalPrice
                FROM Orders
                JOIN [Order Details] USING(OrderID)
                GROUP BY OrderID) USING(OrderID)
            WHERE Orders.CustomerID = 'ALFKI'
                    AND TotalPrice >= 500"""

In [91]:
for row in conn.execute(query3):
    print(row)

(10643, 28, 'Rössle Sauerkraut', 684.0, 1086.0)
(10643, 39, 'Chartreuse verte', 378, 1086.0)
(10643, 46, 'Spegesild', 24, 1086.0)
(10692, 63, 'Vegie-spread', 878.0, 878.0)
(10835, 59, 'Raclette Courdavault', 825, 851)
(10835, 77, 'Original Frankfurter grüne Soße', 26, 851)
(11011, 58, 'Escargots de Bourgogne', 530.0, 960.0)
(11011, 71, 'Flotemysost', 430.0, 960.0)


# MongoDB PART

In [92]:
# Connexion to the database
client = MongoClient()
db = client.Northwind

### Exercise 6.1

In [93]:
# Exercise 6.1

result = {}
for order in db.orders.find({"CustomerID":"ALFKI"}):
    result[order['OrderID']] = {}
    for order_detail in db['order-details'].find({'OrderID':order['OrderID']}):
        for product in db.products.find({'ProductID':order_detail['ProductID']}):
            result[order['OrderID']][product['ProductID']] = product['ProductName']
            
result

{10643: {28: 'Rössle Sauerkraut', 39: 'Chartreuse verte', 46: 'Spegesild'},
 10692: {63: 'Vegie-spread'},
 10702: {3: 'Aniseed Syrup', 76: 'Lakkalikööri'},
 10835: {59: 'Raclette Courdavault', 77: 'Original Frankfurter grüne Soße'},
 10952: {6: "Grandma's Boysenberry Spread", 28: 'Rössle Sauerkraut'},
 11011: {58: 'Escargots de Bourgogne', 71: 'Flotemysost'}}

### Exercise 6.2

In [94]:
# Exercise 6.2

result = {}
for order in db.orders.find({"CustomerID":"ALFKI"}):
    products_in_order = set()
    for order_detail in db['order-details'].find({'OrderID':order['OrderID']}):
            products_in_order.add(order_detail['ProductID'])

    # Count if there are more than 2 different products, if so, store them in result
    # The set type accounts for the uniqueness
    if len(products_in_order)>1:     
        result[order['OrderID']] = products_in_order

# Retrieving the names of the products
final_result = {}
for order, list_products in result.items():
    final_result[order] = {}
    for product in list_products:
        final_result[order][product] = db.products.find_one({'ProductID':product})['ProductName']
final_result

{10643: {28: 'Rössle Sauerkraut', 39: 'Chartreuse verte', 46: 'Spegesild'},
 10702: {3: 'Aniseed Syrup', 76: 'Lakkalikööri'},
 10835: {59: 'Raclette Courdavault', 77: 'Original Frankfurter grüne Soße'},
 10952: {6: "Grandma's Boysenberry Spread", 28: 'Rössle Sauerkraut'},
 11011: {58: 'Escargots de Bourgogne', 71: 'Flotemysost'}}

### Exercise 6.3

In [95]:
# Exercise 6.3

result = {}
for order in db.orders.find({"CustomerID":"ALFKI"}):
    products_in_order = []
    price_per_product = []
    totalPrice = 0
    # Compute the total price for each order and store the information temporarily
    for order_detail in db['order-details'].find({'OrderID':order['OrderID']}):
            products_in_order.append(order_detail['ProductID'])
            price_per_product.append(order_detail['UnitPrice']*order_detail['Quantity'])
            totalPrice += order_detail['UnitPrice']*order_detail['Quantity']

    # If the total price is >=500, then store the information permanently in result
    if totalPrice >= 500:     
        result[order['OrderID']] = {}
        for i in range(len(products_in_order)):
            result[order['OrderID']][products_in_order[i]] = price_per_product[i]
        
        result[order['OrderID']]["TotalPrice"] = totalPrice

result

{10643: {'TotalPrice': 1086.0, 28: 684.0, 46: 24.0, 39: 378.0},
 10692: {'TotalPrice': 878.0, 63: 878.0},
 10835: {'TotalPrice': 851.0, 59: 825.0, 77: 26.0},
 11011: {58: 530.0, 'TotalPrice': 960.0, 71: 430.0}}